In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse, Rectangle
from google.colab import drive

###6.22

###Các nhà nghiên cứu thích thú việc đánh giá chức năng của phổi trong một tổng thể những người không có bệnh lý được yêu cầu chạy trên một máy chạy bộ tới khi kiệt sức. Những mẫu không khí được sưu tập ở một khoảng xác định và thành phần thể khí được phân tích. Kết quả trên 4 phép đo lượng tiêu thụ oxy của 25 nam và 25 nữ được cho bởi bảng 6.12 ở trang 348.

In [2]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
#Import dataset
path = '/content/T6-12.dat.txt'
f = open(path, "r")
data=f.readlines()
male_data=list()
female_data=list()
for txt in data:
  if 'female' in txt:
    female_data.append(txt[:-7])
  else:
    male_data.append(txt[:-6])

male_df=pd.DataFrame(np.loadtxt(male_data))
female_df=pd.DataFrame(np.loadtxt(female_data))

###(a) Tìm sự chênh lệch về giới tính bằng cách kiểm định tính bằng nhau của trung bình từng group. Sử dụng $\alpha=.05$. Nếu bác bỏ $H_0: \boldsymbol{\mu_1}-\boldsymbol{\mu_2}=0$, hãy tìm tổ hợp tuyến tính thoả mãn.

In [4]:
# vectơ trung bình
male_mean = np.array(male_df.apply(np.mean))
print("\nMean vector Male: \n", male_mean)

# Ma trận hiệp phương sai
S_male = np.array(np.cov(male_df.T))
print("\nCovariance matrix Male: \n", S_male)


Mean vector Male: 
 [ 0.3972  5.3296  3.6876 49.4204]

Covariance matrix Male: 
 [[7.12100000e-03 7.00030000e-02 3.14471667e-02 1.50580333e-01]
 [7.00030000e-02 1.14417900e+00 1.47678167e-01 3.43090850e+00]
 [3.14471667e-02 1.47678167e-01 4.55877333e-01 3.30812183e+00]
 [1.50580333e-01 3.43090850e+00 3.30812183e+00 5.52521457e+01]]


In [5]:
# vectơ trung bình
female_mean = np.array(female_df.apply(np.mean))
print("\nMean vector female: \n", female_mean)

# Ma trận hiệp phương sai
S_female = np.array(np.cov(female_df.T))
print("\nCovariance matrix female: \n", S_female)


Mean vector female: 
 [ 0.3136  5.1788  2.3152 38.1548]

Covariance matrix female: 
 [[ 9.73233333e-03  1.54087833e-01  4.16800000e-03  2.97570000e-02]
 [ 1.54087833e-01  2.78066100e+00 -3.94476667e-02  1.28069767e+00]
 [ 4.16800000e-03 -3.94476667e-02  1.20509333e-01  1.09814900e+00]
 [ 2.97570000e-02  1.28069767e+00  1.09814900e+00  2.32608260e+01]]


In [6]:
n_1=n_2=male_df.shape[0]
S_pooled=(n_1-1)/(n_1+n_2-2)*S_male+(n_2-1)/(n_1+n_2-2)*S_female
print(S_pooled)

[[8.42666667e-03 1.12045417e-01 1.78075833e-02 9.01686667e-02]
 [1.12045417e-01 1.96242000e+00 5.41152500e-02 2.35580308e+00]
 [1.78075833e-02 5.41152500e-02 2.88193333e-01 2.20313542e+00]
 [9.01686667e-02 2.35580308e+00 2.20313542e+00 3.92564858e+01]]


In [7]:
t_2=np.matmul((male_mean-female_mean).T,np.linalg.inv((1/n_1+1/n_2)*S_pooled))
t_2=np.matmul(t_2,(male_mean-female_mean))
print('Hotelling T^2: ',t_2)

Hotelling T^2:  96.37322129759951


In [8]:
p=male_df.shape[1]
f = stats.f.ppf(q=1-0.05, dfn=p, dfd=n_1+n_2-p-1)
c_2=(n_1+n_2-2)*p/(n_1+n_2-p-1)*f
print('Critical Value: ',c_2)

Critical Value:  11.002620519729316


Với Hotelling $T^2=96.373 > 11$, ta bác bỏ $H_0$ với mức ý nghĩa $\alpha=0.05$. Ta kết luận là với mức ý nghĩa $95\%$ có sự chênh lệch trong chức năng của phổi giữa 2 giới tính nam và nữ.

In [9]:
linear_combination=np.matmul(np.linalg.inv(S_pooled),(male_mean-female_mean))
print('linear combination most reponsible: ',linear_combination)

linear combination most reponsible:  [99.39897799 -6.3759988  -6.2281408   0.79082376]


###(b) Xây dựng khoảng tin cậy đồng thời $95\%$ cho mỗi $\mu_{1i}-\mu_{2i}, i=1,2,3,4$. So sánh với khoảng tin cậy Bonferroni

In [10]:
for i in range(male_df.shape[1]):
  lowerbound=(male_mean[i]-female_mean[i])-np.sqrt(c_2)*np.sqrt((1/n_1+1/n_2)*S_pooled[i][i])
  upperbound=(male_mean[i]-female_mean[i])+np.sqrt(c_2)*np.sqrt((1/n_1+1/n_2)*S_pooled[i][i])
  print('95% confidence interval for mu_1{}-mu_2{} in range [{}, {}]'.format(i+1,i+1,lowerbound,upperbound))

95% confidence interval for mu_11-mu_21 in range [-0.0025233606309393725, 0.16972336063093962]
95% confidence interval for mu_12-mu_22 in range [-1.1634834568030512, 1.46508345680305]
95% confidence interval for mu_13-mu_23 in range [0.8687428242955314, 1.8760571757044682]
95% confidence interval for mu_14-mu_24 in range [5.387340280917852, 17.143859719082148]


In [11]:
t = stats.t.ppf(q=1-(0.05/(2*p)),df=n_1+n_2-2)
for i in range(male_df.shape[1]):
  lowerbound=(male_mean[i]-female_mean[i])-t*np.sqrt((1/n_1+1/n_2)*S_pooled[i][i])
  upperbound=(male_mean[i]-female_mean[i])+t*np.sqrt((1/n_1+1/n_2)*S_pooled[i][i])
  print('Bonferroni confidence interval for mu_1{}-mu_2{} in range [{},{}]'.format(i+1,i+1,lowerbound,upperbound))

Bonferroni confidence interval for mu_11-mu_21 in range [0.01621483894956366,0.15098516105043658]
Bonferroni confidence interval for mu_12-mu_22 in range [-0.8775296164221205,1.1791296164221192]
Bonferroni confidence interval for mu_13-mu_23 in range [0.978325504411156,1.7664744955888438]
Bonferroni confidence interval for mu_14-mu_24 in range [6.666296454890269,15.86490354510973]


Từ các khoảng tin cậy trên, ta thấy khoảng tin cậy $95\%$ Bonferroni nhỏ hơn so với khoảng tin cậy còn lại.

###(c) Với bộ dữ liệu bảng 6.12 được thu thập từ những tình nguyện viên đại học, do đó, họ không đại diện cho một mẫu ngẫu nhiên. Suy ra điều gì từ thông tin này.

Do dữ liệu bảng 6.12 chỉ được thu thập từ một lượng tình nguyện viên đại học nhỏ và cũng không thể hiện mẫu ngẫu nhiên nên ta không thể mở rộng các kết quả trên cho tổng thể lớn hơn là tổng thể những người đang ở độ tuổi sinh viên đại học.

###6.24


###Các nhà nghiên cứu đã nhận định rằng một thay đổi trong kích thước hộp sọ theo thời gian là bằng chứng cho một giao phối giữa tổng thể bản địa và tổng thể di cư. 4 phép đo được tạo bởi hộp sọ đàn ông Ai Cập trong 3 thời kì khác nhau: thời kì 1 là 4000 năm trước công nguyên, thời kì 2 là 3300 năm trước công nguyên và thời kì 3 là 1850 năm trước công nguyên. Bộ dữ liệu được thể hiện ở bảng 6.13 trang 349. Hãy xây dựng MANOVA 1 chiều của bộ dữ liệu hộp sọ người Ai Cập. Sử dụng $\alpha=.05$. Xây dựng khoảng tin cậy đồng thời $95\%$ để xác định thành phần trung bình nào khác nhau trong các tổng thể được thể hiện bởi 3 thời kì. Giả định MANOVA thông thường có thực tế với những dữ liệu này không ? Giải thích

In [12]:
#Import dữ liệu
path = '/content/T6-13.dat.txt'
data = pd.DataFrame(np.loadtxt(path))
print(data.head(11))

        0      1      2     3    4
0   131.0  138.0   89.0  49.0  1.0
1   125.0  131.0   92.0  48.0  1.0
2   131.0  132.0   99.0  50.0  1.0
3   119.0  132.0   96.0  44.0  1.0
4   136.0  143.0  100.0  54.0  1.0
5   138.0  137.0   89.0  56.0  1.0
6   139.0  130.0  108.0  48.0  1.0
7   125.0  136.0   93.0  48.0  1.0
8   131.0  134.0  102.0  51.0  1.0
9   134.0  134.0   99.0  51.0  1.0
10  129.0  138.0   95.0  50.0  1.0


In [13]:
list_mean=list()
list_data=list()
list_cov=list()

#Trung bình và ma trận hiệp phương sai của hộp sọ trong thời kì 1
data_1=data[data.iloc[:,-1]==1]
data_1=data_1.drop([4],axis=1)
list_data.append(data_1)
list_cov.append(np.cov(data_1.T))

x_1_mean=np.mean(data_1,axis=0)
list_mean.append(x_1_mean.to_numpy().reshape(-1,1))

#Trung bình và ma trận hiệp phương sai của hộp sọ trong thời kì 2
data_2=data[data.iloc[:,-1]==2]
data_2=data_2.drop([4],axis=1)
list_data.append(data_2)
list_cov.append(np.cov(data_2.T))

x_2_mean=np.mean(data_2,axis=0)
list_mean.append(x_2_mean.to_numpy().reshape(-1,1))

#Trung bình và ma trận hiệp phương sai của hộp sọ trong thời kì 3
data_3=data[data.iloc[:,-1]==3]
data_3=data_3.drop([4],axis=1)
list_data.append(data_3)
list_cov.append(np.cov(data_3.T))

x_3_mean=np.mean(data_3,axis=0)
list_mean.append(x_3_mean.to_numpy().reshape(-1,1))

data_all=data.copy()
data_all=data_all.drop([4],axis=1)
x_all_mean=np.mean(data_all,axis=0).to_numpy().reshape(-1,1)



In [14]:
#Tính B
B=np.zeros([x_1_mean.shape[0],x_1_mean.shape[0]])
for i in range(0,3):
  B+=list_data[i].shape[0]*np.matmul((list_mean[i]-x_all_mean),(list_mean[i]-x_all_mean).T)

print(B)

[[ 150.2          20.3        -161.83333333    5.03333333]
 [  20.3          20.6         -38.73333333    6.43333333]
 [-161.83333333  -38.73333333  190.28888889  -10.85555556]
 [   5.03333333    6.43333333  -10.85555556    2.02222222]]


In [15]:
#Tính W
W=np.zeros([x_1_mean.shape[0],x_1_mean.shape[0]])
for i in range(0,2):
  W+=(list_data[i].shape[0]-1)*list_cov[i]

print(W)

[[1433.93333333  149.7         151.43333333  263.56666667]
 [ 149.7        1205.5          74.6         174.5       ]
 [ 151.43333333   74.6        1552.03333333  -50.13333333]
 [ 263.56666667  174.5         -50.13333333  474.83333333]]


In [16]:
#Tính hệ số wilk lambda
wilk_lambda=(np.linalg.det(W))/(np.linalg.det(B+W))
print(wilk_lambda)

0.7730783604911525


In [17]:
#Tính giá trị
p=4
value=((data.shape[0]-p-2)/p)*((1-np.sqrt(wilk_lambda))/np.sqrt(wilk_lambda))
print('Calculated value: ',value)

Calculated value:  2.884026053800916


In [18]:
#Tính giá trị tới hạn
f=stats.f.ppf(q=1-0.05,dfn=2*p,dfd=2*(data.shape[0]-p-2))
print('Critical value: ',f)

Critical value:  1.9938838709988889


Ta nhận thấy 2.884 > 1.99388. Nghĩa là: $(\dfrac{\sum n_l-p-2}{p})(\dfrac{1-\sqrt{\Lambda*}}{\sqrt{\Lambda*}}) > F_{2p,2(\sum n_l-p-2)}(\alpha)$. Nên ta bác bỏ $H_0$ với mức ý nghĩa $95\%$. Nghĩa là có sự thay đổi trong kích thước hộp sọ ở 3 thời kì

In [19]:
#Tính khoảng tin cậy 95%
t=stats.t.ppf(q=1-0.05/(p*3*2),df=data.shape[0]-3)
for i in range(0,4):
  for l in range(0,3):
    for k in range(l+1,3):
      lower=list_mean[k][i]-list_mean[l][i]-t*np.sqrt(1/(data.shape[0]-3)*(1/list_data[k].shape[0]+1/list_data[l].shape[0])*W[i][i])
      upper=list_mean[k][i]-list_mean[l][i]+t*np.sqrt(1/(data.shape[0]-3)*(1/list_data[k].shape[0]+1/list_data[l].shape[0])*W[i][i])
      print('95% Confidence interval of tau_{}{} - tau_{}{}: [{}, {}]'.format(k,i,l,i,lower,upper))


95% Confidence interval of tau_10 - tau_00: [[-2.08494115], [4.08494115]]
95% Confidence interval of tau_20 - tau_00: [[0.01505885], [6.18494115]]
95% Confidence interval of tau_20 - tau_10: [[-0.98494115], [5.18494115]]
95% Confidence interval of tau_11 - tau_01: [[-3.72856401], [1.92856401]]
95% Confidence interval of tau_21 - tau_01: [[-2.62856401], [3.02856401]]
95% Confidence interval of tau_21 - tau_11: [[-1.72856401], [3.92856401]]
95% Confidence interval of tau_12 - tau_02: [[-3.30946708], [3.10946708]]
95% Confidence interval of tau_22 - tau_02: [[-6.34280042], [0.07613375]]
95% Confidence interval of tau_22 - tau_12: [[-6.24280042], [0.17613375]]
95% Confidence interval of tau_13 - tau_03: [[-2.07522336], [1.47522336]]
95% Confidence interval of tau_23 - tau_03: [[-1.74189002], [1.80855669]]
95% Confidence interval of tau_23 - tau_13: [[-1.44189002], [2.10855669]]


Ta nhận thấy có khoảng tin cậy $95\%$ ở biến 0 là độ rộng tối đa của hộp sọ không có chứa 0. Đây chính là thành phần có trung bình mà khác nhau trong các tổng thể qua các thời kì.
Để kiểm tra giả định MANOVA ta sẽ dùng kiểm định Box để kiểm định tính bằng nhau của các ma trận hiệp phương sai với mức ý nghĩa $5\%$

In [20]:
#Box Test cho kiểm định tính bằng nhau của ma trận hiệp phương sai với mức ý nghĩa 95%
n_1,n_2,n_3=data_1.shape[0],data_2.shape[0],data_3.shape[0]
p=4
g=3
u=(1/(n_1-1)+1/(n_2-1)+1/(n_3-1)-1/(n_1+n_2+n_3-3))*((2*p*p+3*p-1)/(6*(p+1)*(g-1)))
S_pooled=1/(n_1+n_2+n_3-3)*((n_1-1)*list_cov[0]+(n_2-1)*list_cov[1]+(n_3-1)*list_cov[2])
C=(1-u)*((n_1+n_2+n_3-3)*np.log(np.linalg.det(S_pooled))-(n_1-1)*np.log(np.linalg.det(list_cov[0]))-(n_2-1)*np.log(np.linalg.det(list_cov[1]))-(n_3-1)*np.log(np.linalg.det(list_cov[2])))
print('C value: ',C)
chi=stats.chi2.ppf(q=1-0.05,df=p*(p+1)*(g-1)/2)
print('Critical value chi square: ',chi)

C value:  21.048436386915938
Critical value chi square:  31.410432844230918


Ta thấy $C=21.04 < 31.414=\chi^2_{p(p+1)(g-1)/2}(\alpha)$ Nên ta không đủ cơ sở bác bỏ $H_0$. Do đó, với mức ý nghĩa $\alpha=.05$, ta kết luận các ma trận hiệp phương sai của các tổng thể là bằng nhau. Do đó, giả đinh MANOVA thông thường là phù hợp với bộ dữ liệu này

###6.30

###Bảng 6.16 ở trang 353 chứa thông tin về thành phần trong các quặng khoáng sản của xương. Với 24 đối tượng đầu tiên là bảng 1.8, 1 năm sau sự tham gia của họ vào một chương trình thí nghiệm. So sánh dữ liệu từ hai bảng và xác định xem có mất mát xương không ?

###(a) Kiểm định với $\alpha=.05$

In [21]:
path = '/content/T6-16.dat.txt'
bone_after = pd.DataFrame(np.loadtxt(path))
print(bone_after.head(11))
print(bone_after.shape)

        0      1      2      3      4      5
0   1.027  1.051  2.268  2.246  0.869  0.964
1   0.857  0.817  1.718  1.710  0.602  0.689
2   0.875  0.880  1.953  1.756  0.765  0.738
3   0.873  0.698  1.668  1.443  0.761  0.698
4   0.811  0.813  1.643  1.661  0.551  0.619
5   0.640  0.734  1.396  1.378  0.753  0.515
6   0.947  0.865  1.851  1.686  0.708  0.787
7   0.886  0.806  1.742  1.815  0.687  0.715
8   0.991  0.923  1.931  1.776  0.844  0.656
9   0.977  0.925  1.933  2.106  0.869  0.789
10  0.825  0.826  1.609  1.651  0.654  0.726
(24, 6)


In [22]:
path = '/content/T1-8.DAT.txt'
bone_before = pd.DataFrame(np.loadtxt(path))
bone_before=bone_before.iloc[:24,:]
print(bone_before.head(11))
print(bone_before.shape)

        0      1      2      3      4      5
0   1.103  1.052  2.139  2.238  0.873  0.872
1   0.842  0.859  1.873  1.741  0.590  0.744
2   0.925  0.873  1.887  1.809  0.767  0.713
3   0.857  0.744  1.739  1.547  0.706  0.674
4   0.795  0.809  1.734  1.715  0.549  0.654
5   0.787  0.779  1.509  1.474  0.782  0.571
6   0.933  0.880  1.695  1.656  0.737  0.803
7   0.799  0.851  1.740  1.777  0.618  0.682
8   0.945  0.876  1.811  1.759  0.853  0.777
9   0.921  0.906  1.954  2.009  0.823  0.765
10  0.792  0.825  1.624  1.657  0.686  0.668
(24, 6)


In [23]:
# vectơ trung bình
bone_before_mean = np.array(bone_before.apply(np.mean))
print("\nMean vector Mineral Content Before: \n", bone_before_mean)

# Ma trận hiệp phương sai
S_before = np.array(np.cov(bone_before.T))
print("\nCovariance matrix Mineral Content Before: \n", S_before)

# vectơ trung bình
bone_after_mean = np.array(bone_after.apply(np.mean))
print("\nMean vector Mineral Content After: \n", bone_after_mean)

# Ma trận hiệp phương sai
S_after = np.array(np.cov(bone_after.T))
print("\nCovariance matrix Male: \n", S_after)


Mean vector Mineral Content Before: 
 [0.84083333 0.81341667 1.78525    1.72925    0.69754167 0.68658333]

Covariance matrix Mineral Content Before: 
 [[0.01333728 0.0104502  0.0227467  0.02052635 0.00898649 0.00774223]
 [0.0104502  0.01128712 0.01839067 0.02130185 0.00802337 0.00836762]
 [0.0227467  0.01839067 0.08241089 0.06859602 0.01623964 0.01199907]
 [0.02052635 0.02130185 0.06859602 0.07169037 0.01750647 0.01646554]
 [0.00898649 0.00802337 0.01623964 0.01750647 0.01084435 0.00712376]
 [0.00774223 0.00836762 0.01199907 0.01646554 0.00712376 0.00968625]]

Mean vector Mineral Content After: 
 [0.84095833 0.81016667 1.77808333 1.71691667 0.71266667 0.68675   ]

Covariance matrix Male: 
 [[0.01551526 0.01116144 0.02949613 0.02415421 0.00897607 0.00928429]
 [0.01116144 0.01175754 0.02348533 0.02357567 0.00868114 0.0096873 ]
 [0.02949613 0.02348533 0.10505521 0.08406183 0.02396133 0.01832702]
 [0.02415421 0.02357567 0.08406183 0.0843433  0.02121584 0.01965733]
 [0.00897607 0.00868114 

In [24]:
n_1=n_2=24
S_pooled=(n_1-1)/(n_1+n_2-2)*S_before+(n_2-1)/(n_1+n_2-2)*S_after
print(S_pooled)

[[0.01442627 0.01080582 0.02612141 0.02234028 0.00898128 0.00851326]
 [0.01080582 0.01152233 0.020938   0.02243876 0.00835226 0.00902746]
 [0.02612141 0.020938   0.09373305 0.07632893 0.02010049 0.01516304]
 [0.02234028 0.02243876 0.07632893 0.07801683 0.01936115 0.01806143]
 [0.00898128 0.00835226 0.02010049 0.01936115 0.01117594 0.00733347]
 [0.00851326 0.00902746 0.01516304 0.01806143 0.00733347 0.01111622]]


In [25]:
t_2=np.matmul((bone_after_mean-bone_before_mean).T,np.linalg.inv((1/n_1+1/n_2)*S_pooled))
t_2=np.matmul(t_2,(bone_after_mean-bone_before_mean))
print('Hotelling T^2: ',t_2)

Hotelling T^2:  0.8299432210795187


In [26]:
p=6
f = stats.f.ppf(q=1-0.05, dfn=p, dfd=n_1+n_2-p-1)
c_2=(n_1+n_2-2)*p/(n_1+n_2-p-1)*f
print('Critical Value: ',c_2)

Critical Value:  15.683337743007508


Ta thấy Hotelling 0.829 < 15.683 nên ta không đủ cơ sở để bác bỏ $H_0$ với mức ý nghĩa $5\%$. Ta kết luận các thành phần của xương không bị mất mát 

###(b) Xây dựng khoảng tin cậy đồng thời $95\%$ cho sự chênh lệch trung bình

In [27]:
for i in range(6):
  lowerbound=(bone_after_mean[i]-bone_before_mean[i])-np.sqrt(c_2)*np.sqrt((1/n_1+1/n_2)*S_pooled[i][i])
  upperbound=(bone_after_mean[i]-bone_before_mean[i])+np.sqrt(c_2)*np.sqrt((1/n_1+1/n_2)*S_pooled[i][i])
  print('95% confidence interval for mu_1{}-mu_2{} in range [{}, {}]'.format(i+1,i+1,lowerbound,upperbound))

95% confidence interval for mu_11-mu_21 in range [-0.13718608872058177, 0.13743608872058163]
95% confidence interval for mu_12-mu_22 in range [-0.12596531703856026, 0.11946531703856053]
95% confidence interval for mu_13-mu_23 in range [-0.35717227283569053, 0.34283893950235655]
95% confidence interval for mu_14-mu_24 in range [-0.33165087993423037, 0.30698421326756375]
95% confidence interval for mu_15-mu_25 in range [-0.10573168483474202, 0.13598168483474235]
95% confidence interval for mu_16-mu_26 in range [-0.12036669849968282, 0.12070003183301634]


###(c) Xây dựng khoảng tin cậy $95\%$ Bonferroni và so sánh khoảng này với khoảng ở phần b

In [28]:
t = stats.t.ppf(q=1-(0.05/(2*p)),df=n_1+n_2-2)
for i in range(6):
  lowerbound=(bone_after_mean[i]-bone_before_mean[i])-t*np.sqrt((1/n_1+1/n_2)*S_pooled[i][i])
  upperbound=(bone_after_mean[i]-bone_before_mean[i])+t*np.sqrt((1/n_1+1/n_2)*S_pooled[i][i])
  print('Bonferroni confidence interval for mu_1{}-mu_2{} in range [{},{}]'.format(i+1,i+1,lowerbound,upperbound))

Bonferroni confidence interval for mu_11-mu_21 in range [-0.09547340556508102,0.09572340556508088]
Bonferroni confidence interval for mu_12-mu_22 in range [-0.08868657148602403,0.0821865714860243]
Bonferroni confidence interval for mu_13-mu_23 in range [-0.2508467525240225,0.23651341919068852]
Bonferroni confidence interval for mu_14-mu_24 in range [-0.23464785001536137,0.20998118334869476]
Bonferroni confidence interval for mu_15-mu_25 in range [-0.06901755891302268,0.09926755891302301]
Bonferroni confidence interval for mu_16-mu_26 in range [-0.08375079138178526,0.08408412471511878]


Từ các khoảng tin cậy trên, ta thấy khoảng tin cậy Bonferroni hẹp hơn so với khoảng tin cậy còn lại.